# Password Cracker Using a Dictionary Attack For The UNIX File System 
By Heidi Nguyen

## Background
    - UNIX stores all passwords in the file /etc/passwd. Well, it doesn’t store the password itself. Instead, it stores a signature of the password by using the password to encrypt a block of zero bits prepended by a salt value with a one-way function called crypt( ). The result of the crypt( ) function is stored in the /etc/passwd file. For example, for a password of “egg” and salt equal to “HX”, the function crypt(‘egg’,‘HX’) returns HX9LLTdc/jiDE.
    - When you try to log in, the program /bin/login takes the password that you typed, uses crypt( ) to encrypt a block of zero bits, and compares the result of this function with the value stored in the /etc/passwd file.
    - The security of this approach rests on both the strength of the crypt( ) function and the difficulty in guessing a user’s password. The crypt( ) algorithm has proven to be highly resistant to attacks. Conversely, the user’s choices for passwords have been found to be relatively easy to guess, with many passwords being words contained in the dictionary.

## The Original Unix crypt Password Mechanism: [1]
-	Read the first eight characters of the password (the rest is ignored).  If a shorter password is used, pad with zeros.   Using 7-bit ASCII results in a 56 bit partial key.
-	Next a 12 bit random value is chosen and encoded to form a salt, which has two character string from the ASCII letters and digits. Often the current time is used to generate the salt.
-	 The salt is added to the partial key to generate the longer key that is used with DES encryption.
-	 A message (a string of eight bytes initialized to all zeros) is then encrypted with DES using the key.  This results in 64 bits (eight bytes) of encrypted output.
-	The encrypted output is used as the input message (in place of the zeros) and the encryption is repeated, for a total of 25 DES encryption cycles.  (Although DES encryption is used, the original message is known and thus this is more of a hash of the password than an encryption of the all zero message.)
-	 The 64 bit result is then encoded to be all printable characters (letters and digits and periods only) that results in an 11 character ASCII string.
-	The 2-character salt plus the 11 character encrypted text is stored in the password file.
-	To validate a user's password, the entered password plus the retrieved salt value is used to generate the 11character result.  This result is compared, byte for byte, with the copy in the password file.
    

## Method

A simple dictionary attack involves computing the possible signatures generated for each word in the dictionary with a range of salt values. 

    - We will need to use the crypt( ) algorithm, the crypt library, that hashes UNIX passwords. Note that the crypt library already exists in the Python 2.79 standard library (on UNIX-based operating systems). (Note: for Windows- based operating systems, you will need to find the correct way to import the UNIX crypt() algorithm.) 
    - To calculate the encrypted UNIX password signature, we simply call the function crypt.cypt( ) and pass it the password and salt as parameters. This function returns the signature as a string. 
    - Start your program by reading in the passwords.txt file and, for each password found in the file, iterate through each dictionary word found in the dictionary.txt file and appropriate salt value.
    - Report out the password found, if any, for each user. If no password is found, indicate that no password was found. 2) Using literature review, identify from where you can retrieve the salt value used in generating the signature.


In [6]:
# look at the files
# load in the images 
%cat'passwords.txt'
%cat'dictionary.txt'

victim: HX9LLTdc/jiDE: 503:100:Iama Victim:/home/victim:/bin/sh
root: DFNFxgW7C05fo: 504:100: Markus Hess:/root:/bin/bash
apple
orange
egg
lemon
grapes
secret
strawberry
password


In [2]:
#libraries
from PIL import Image
import sys, os, random, struct, hashlib, time
import crypt

# We started by defining a function called testpass that take the pw variable called (CryptPass) 
# and compared it with each word in dictfile
def test_pass(CryptPass, dictfile):
    salt = CryptPass[0:2] # make sure that the salt is the first 2 characters of the cryptPass
    # open the dictionary file
    dict_file = open(dictfile, 'r')
    # For each word in in the dictionary we want to know it's a new line.
    # So it wont encrypt all the words in one, we do that by using ('\n')
    for word in dict_file.readlines():
        #word = word.split("\n")
        word = word.rstrip() # to remove all trailing space at the end
        # Next we define a new variable called Cryptword and give it the data :
        # crypt.crypt(word,salt) is encrypting the words in the dict with salt that is found in the CryptPass[0:2]
        CryptWord  = crypt.crypt(word,salt)
        # Now if the CryptWord is equal ('==') to the CryptPass,
        # that means (the hashed word in dictionary and the salt) is the same as the passwd in the passfile.
        # Then it means that the password has been found.
        if CryptWord == CryptPass:
            print "pass found"
            print "pass is: " + word
            print "CryptWord with salt is: " + CryptWord
            return True # you can return True or just Return so it doesn't continue going thru the dict file. because you already founded the pass!
        #         elif crypt.crypt(dict_file.readlines()[],salt) :
        else:
            continue # continue to the next word in the dict
    return False # in the case there is no pass found, function return False

# Here we have defined a second function called Main():
# This is how /etc/shadow file look like:
# Username or victim: HX9LLTdc/jiDE: 503:100:Iama Victim:/home/victim:/bin/sh
#               root: DFNFxgW7C05fo: 504:100: Markus Hess:/root:/bin/bash

def main(pass_file,dictfile):
    #     opens the Passfile in read only mode.
    passfile = open(pass_file, 'r')
    #     it's going to read each line in the passfile.
    for line in passfile.readlines():
        #     Whenever there is an colon ": " in the line.
        #     You want to save the first word before the colon in the variable "user" or "victim".
        #     After splitting by ":", since the user is always before the colon --> [0] item in the list, and the hashed password is always after the colon,
        #     so I'm going to grab the [1] item in the list
        if ":" in line: # for general application, it should be just ":" or "Username:"
            user = line.split(":")[0] # splitting the username or victim and the password by the colon ":"
            cryptPass = line.split(":")[1].strip(' ') # the .split(' ') to make sure that there aren't any spaces between them.
            print "cracking password for: " + user
            # call test_pass()
            test_pass(cryptPass,dictfile)
            found_pass = test_pass(cryptPass,dictfile) # when there is pass found, test_pass() return True
            
            # print when there is no pass found, where test_pass() return False
            if found_pass == False:
                print "pass not found"

# calling functions
if __name__ == '__main__':
    start = time.time()
    # Set wd
    path = os.chdir("/Users/heidinguyen/Downloads/")
#    path = "/Volumes/NO NAME 1/Data Science/2018-0106 MSDS 7349 Data and Network Security/Unit 13 - Quiz 12 & HW 3/HW 3/"
    #call main()
    pass_file = 'passwords.txt'
    dictfile = 'dictionary.txt'
    main(pass_file,dictfile)
    print 'It took', time.time()-start, 'seconds.'



cracking password for: victim
pass found
pass is: egg
CryptWord with salt is: HX9LLTdc/jiDE
pass found
pass is: egg
CryptWord with salt is: HX9LLTdc/jiDE
cracking password for: root
pass not found
It took 0.00827813148499 seconds.


Reference: 
    
    1. Violent Python: A Cookbook for Hackers, Forensic Analysts, Penetration Testers and Security Engineers by TJ O'Connor
    2. https://github.com/shadow-box/Violent-Python-Examples/tree/master/Chapter-1
    3. http://stackoverflow.com/questions/26021982/python-crypt-equivalent-for-windows
    4. https://pythonhosted.org/passlib/lib/passlib.hash.des_crypt.html
